## Extracting raw data

In [ ]:

## Environment setup
## Reference - https://github.com/int-brain-lab/iblenv
!git clone https://github.com/int-brain-lab/iblapps
!pip install --editable iblapps
!git clone https://github.com/int-brain-lab/iblenv

In [ ]:
cd iblenv
!pip install --requirement requirements.txt

In [4]:
import datajoint as dj

dj.config['database.host'] = 'datajoint-public.internationalbrainlab.org'
dj.config['database.user'] = 'ibl-public'
dj.config['database.password'] = 'ibl-public'
dj.conn() # explicitly checks if database connection can be established

[2023-07-18 16:51:48,444][INFO]: Connecting ibl-public@datajoint-public.internationalbrainlab.org:3306
[2023-07-18 16:51:49,939][INFO]: Connected ibl-public@datajoint-public.internationalbrainlab.org:3306


DataJoint connection (connected) ibl-public@datajoint-public.internationalbrainlab.org:3306

In [5]:
from one.api import ONE

ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')
one

One (online, https://openalyx.internationalbrainlab.org)

In [6]:
from nma_ibl import behavior, data, acquisition, subject

#### Extracting wheel
References :

1) https://int-brain-lab.github.io/iblenv/notebooks_external/one_quickstart.html
2) https://int-brain-lab.github.io/iblenv/notebooks_external/docs_wheel_moves.html
3) https://int-brain-lab.github.io/iblenv/notebooks_external/loading_wheel_data.html


In [43]:
eids, info = one.search(date_range=['2018-08-01', '2020-08-31'], dataset='wheel', details=True)

from pprint import pprint
pprint(eids)
pprint(info)

['ebe2efe3-e8a1-451a-8947-76ef42427cc9',
 'b69b86be-af7d-4ecf-8cbf-0cd356afa1bd',
 'edd22318-216c-44ff-bc24-49ce8be78374',
 '71e55bfe-5a3a-4cba-bdc7-f085140d798e',
 '626126d5-eecf-4e9b-900e-ec29a17ece07',
 '49e0ab27-827a-4c91-bcaa-97eea27a1b8d',
 '81a78eac-9d36-4f90-a73a-7eb3ad7f770b',
 '5adab0b7-dfd0-467d-b09d-43cb7ca5d59c',
 'e56541a5-a6d5-4750-b1fe-f6b5257bfe7c',
 '5d01d14e-aced-4465-8f8e-9a1c674f62ec',
 '7f6b86f9-879a-4ea2-8531-294a221af5d0',
 '8c33abef-3d3e-4d42-9f27-445e9def08f9',
 'c557324b-b95d-414c-888f-6ee1329a2329',
 '61e11a11-ab65-48fb-ae08-3cb80662e5d6',
 'c7248e09-8c0d-40f2-9eb4-700a8973d8c8',
 '280ee768-f7b8-4c6c-9ea0-48ca75d6b6f3',
 'ff48aa1d-ef30-4903-ac34-8c41b738c1b9',
 '03063955-2523-47bd-ae57-f7489dd40f15',
 '1e45d992-c356-40e1-9be1-a506d944896f',
 '4d8c7767-981c-4347-8e5e-5d5fffe38534',
 '6274dda8-3a59-4aa1-95f8-a8a549c46a26',
 '41872d7f-75cb-4445-bb1a-132b354c44f0',
 'fe1fd79f-b051-411f-a0a9-2530a02cc78d',
 '934dd7a4-fbdc-459c-8830-04fe9033bc28',
 '549caacc-3bd7-

In [45]:
#one = ONE()

#--------Define eid-----------#
eid = eids[0]
eid


'ebe2efe3-e8a1-451a-8947-76ef42427cc9'

In [46]:
one.list_datasets(eid, collection='alf')


['alf/_ibl_bodyCamera.dlc.pqt',
 'alf/_ibl_bodyCamera.times.npy',
 'alf/_ibl_leftCamera.dlc.pqt',
 'alf/_ibl_leftCamera.features.pqt',
 'alf/_ibl_leftCamera.times.npy',
 'alf/_ibl_rightCamera.dlc.pqt',
 'alf/_ibl_rightCamera.features.pqt',
 'alf/_ibl_rightCamera.times.npy',
 'alf/_ibl_trials.goCueTrigger_times.npy',
 'alf/_ibl_trials.stimOff_times.npy',
 'alf/_ibl_trials.table.pqt',
 'alf/_ibl_wheel.position.npy',
 'alf/_ibl_wheel.timestamps.npy',
 'alf/_ibl_wheelMoves.intervals.npy',
 'alf/_ibl_wheelMoves.peakAmplitude.npy',
 'alf/bodyCamera.ROIMotionEnergy.npy',
 'alf/bodyROIMotionEnergy.position.npy',
 'alf/leftCamera.ROIMotionEnergy.npy',
 'alf/leftROIMotionEnergy.position.npy',
 'alf/licks.times.npy',
 'alf/probes.description.json',
 'alf/rightCamera.ROIMotionEnergy.npy',
 'alf/rightROIMotionEnergy.position.npy']

In [32]:
mouse_session_ids = (acquisition.Session * subject.Subject).fetch(
                            format = "frame").reset_index(level = 1)["session_uuid"].to_dict()

In [13]:
#-------Example of how to get an eid for a specific session and mouse------
key = {'subject_uuid': '00778394-c956-408d-8a6c-ca3b05a611d5',
       'session_uuid': 'fb9bdf18-76be-452b-ac4e-21d5de3a6f9f'}

# has to be 36 characters long
#eid = str((acquisition.Session & key).fetch1('session_uuid'))
#eid

'fb9bdf18-76be-452b-ac4e-21d5de3a6f9f'

In [35]:
keys = [{"subject_uuid": k, "session_uuid": v} for k, v in mouse_session_ids.items()]
keys[0]

{'subject_uuid': UUID('00778394-c956-408d-8a6c-ca3b05a611d5'),
 'session_uuid': UUID('8b960a4a-a53e-4d95-b2eb-82fade80e6a6')}

In [39]:
#eid = str((acquisition.Session & keys[0]).fetch1('session_uuid'))
#eid, len(eid)

('8b960a4a-a53e-4d95-b2eb-82fade80e6a6', 36)

In [47]:
print(one.eid2ref(eid, as_dict=False))

2020-08-19_1_NYU-21


In [48]:
wheel = one.load_object(eid, 'wheel', collection='alf')
wheelMoves = one.load_object(eid, 'wheelMoves', collection='alf')
wheel, wheelMoves

C:\Users\pierc\Downloads\ONE\openalyx.internationalbrainlab.org\angelakilab\Subjects\NYU-21\2020-08-19\001\alf\_ibl_wheel.timestamps.npy: 100%|██████████| 14.4M/14.4M [00:03<00:00, 3.95MB/s]
C:\Users\pierc\Downloads\ONE\openalyx.internationalbrainlab.org\angelakilab\Subjects\NYU-21\2020-08-19\001\alf\_ibl_wheel.position.npy: 100%|██████████| 14.4M/14.4M [00:02<00:00, 5.36MB/s]
C:\Users\pierc\Downloads\ONE\openalyx.internationalbrainlab.org\angelakilab\Subjects\NYU-21\2020-08-19\001\alf\_ibl_wheelMoves.peakAmplitude.npy: 100%|██████████| 16.4k/16.4k [00:00<00:00, 46.0kB/s]
C:\Users\pierc\Downloads\ONE\openalyx.internationalbrainlab.org\angelakilab\Subjects\NYU-21\2020-08-19\001\alf\_ibl_wheelMoves.intervals.npy: 100%|██████████| 32.7k/32.7k [00:00<00:00, 108kB/s]


({'timestamps': array([3.87841501e-02, 7.44962883e-02, 1.00896390e-01, ...,
         4.66707687e+03, 4.66712132e+03, 4.66719256e+03]),
  'position': array([-1.53398079e-03, -3.06796158e-03, -4.60194236e-03, ...,
         -7.01613667e+02, -7.01615201e+02, -7.01613667e+02])},
 {'peakAmplitude': array([-2.18049452, -0.42340395,  0.69538343, ...,  1.16750643,
         -0.40233198,  0.4000457 ]),
  'intervals': array([[2.55578415e+00, 3.93578415e+00],
         [4.23378415e+00, 4.58978415e+00],
         [6.61778415e+00, 6.91378415e+00],
         ...,
         [4.65741878e+03, 4.65951778e+03],
         [4.66506078e+03, 4.66565978e+03],
         [4.66605878e+03, 4.66652078e+03]])})

#### Loading video tracking data

References:

https://int-brain-lab.github.io/iblenv/notebooks_external/loading_video_data.html
https://docs.google.com/document/d/1OqIqqakPakHXRAwceYLwFY9gOrm8_P62XIfCTnHwstg/edit#heading=h.vt7n2cynbag7


In [ ]:
label = 'right' # Camera ID from 3 options 'left', 'right' or 'body'
video_features = one.load_object(eid, f'{label}Camera', collection='alf')
video_features.keys()

In [ ]:
#-------Face tracking with DLC----
video_features["dlc"]

#### Loading lick data

In [ ]:
licks = one.load_object(eid, 'licks', collection='alf')
trials = one.load_object(eid, 'trials', collection='alf')